In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
import re
from scipy.stats import fisher_exact
import math


In [2]:
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
import warnings

# Ignore specific warning
warnings.filterwarnings("ignore", category=RuntimeWarning)


# **1. Read Files**

## **1.1 GNOMAD**

In [9]:
gnomad = pd.read_pickle("../data/pipeline_results/gnomad_variants.pkl")
len(gnomad)

15941

In [10]:
gnomad['#CHROM'] = gnomad['#CHROM'].str.replace('chr', '')


In [11]:
gnomad.SYMBOL.value_counts()

SYMBOL
HLA-A       59
AHNAK2      57
HLA-B       53
HLA-C       51
HLA-DQB1    47
            ..
NEFM         1
ADAMDEC1     1
ADAM28       1
ENTPD4       1
RBMY1J       1
Name: count, Length: 7188, dtype: int64

## **1.2 CLINVAR**

In [12]:
clinvar = pd.read_pickle("../data/pipeline_results/clinvar_variants.pkl")
len(clinvar)

31252

In [13]:
len(clinvar)

31252

## **1.3 PAVS**

In [14]:
pavs = pd.read_pickle("../data/pipeline_results/pavs_variants.pkl")
len(pavs)

481

# **2. Concatenate**

In [15]:
# Add a 'Source' column to each dataframe
clinvar['Source'] = 'Pathogenic'
gnomad['Source'] = 'Common'
pavs['Source'] = 'Pavs'

In [16]:
concatenated_df = pd.concat([clinvar, gnomad,pavs], ignore_index=True)
concatenated_df

,#CHROM,POS,ID,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,ALLELEID,CLNDN,CLNDNINCL,CLNDISDB,CLNDISDBINCL,CLNHGVS,CLNREVSTAT,CLNSIG,CLNSIGCONF,CLNSIGINCL,CLNVC,CLNVCSO,CLNVI,DBVARID,MC,ORIGIN,RS,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,RefSeq,SIFT,PolyPhen,DOMAINS,CLIN_SIG,SOMATIC,PHENO,Conservation,BLOSUM62,GENEINFO_NAME,GENEINFO_ID,RefSeq_noversion,UniProt_IDs,PDB_path,Residue_position,Residue,DSSP_path,intra_contacts,is_catalytic,pLDDT,secondary_structure,accessibility,total energy,Backbone Hbond,Sidechain Hbond,Van der Waals,Electrostatics,Solvation Polar,Solvation Hydrophobic,Van der Waals clashes,entropy sidechain,entropy mainchain,sloop_entropy,mloop_entropy,cis_bond,torsional clash,backbone clash,helix dipole,water bridge,disulfide,electrostatic kon,partial covalent bonds,energy Ionisation,Entropy Complex,Source,AC,AF,AN,QD,variant_type,vep,flags,hgvsAllele,geneSymbol,protein,zygosityLabel,phenoLabels,sex,pavs_mutation,vep_mutation
0,1,197429459,1048144,G,C,NaN,NaN,NaN,1036051,Leber_congenital_amaurosis_8,NaN,"MONDO:MONDO:0013453,MedGen:C3151202,OMIM:61383...",NaN,NC_000001.11:g.197429459G>C,"criteria_provided,_single_submitter",Likely_pathogenic,NaN,NaN,single_nucleotide_variant,SO:0001483,NaN,NaN,"SO:0001583|missense_variant,SO:0001619|non-cod...",0,1571544334,C,missense_variant,MODERATE,CRB1,ENSG00000134376,Transcript,ENST00000367400,protein_coding,8/12,NaN,NaN,NaN,2848/4958,2687/4221,896/1406,C/S,tGt/tCt,CM1610693,NaN,1,SNV,HGNC,HGNC:2343,YES,NM_201253.3,NaN,ENSP00000356370,P82279.210,A0A7D6VM04.8,UPI0000073345,P82279-1,NM_201253.3,deleterious(0),probably_damaging(0.947),Gene3D:2.10.25.10&AFDB-ENSP_mappings:AF-P82279...,NaN,NaN,1,2.050,-1,CRB1,23418,NM_201253,"[P82279-1, A0A7D6VM04]",/ibex/scratch/projects/c2102/databases/alphafo...,896,Cys,/ibex/scratch/projects/c2102/databases/dssp/al...,"[902, 903, 909, 910, 911, 912, 920, 921, 922]",False,78.820000,NaN,23,5.248440,0.073737,0.000000e+00,0.751425,0.000000e+00,0.134081,1.298280,0.956765,-0.603521,-0.434252,0.0,0.0,0.0,0.113566,0.006548,0.000000e+00,0.0,2.95836,0.0,0.0,0.000000,0.0,Pathogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,197429466,1213987,T,G,NaN,NaN,NaN,1203980,Retinitis_pigmentosa_12,NaN,"MONDO:MONDO:0010818,MedGen:C1838647,OMIM:60010...",NaN,NC_000001.11:g.197429466T>G,"criteria_provided,_single_submitter",Likely_pathogenic,NaN,NaN,single_nucleotide_variant,SO:0001483,NaN,NaN,"SO:0001583|missense_variant,SO:0001619|non-cod...",1,NaN,G,missense_variant,MODERATE,CRB1,ENSG00000134376,Transcript,ENST00000367400,protein_coding,8/12,NaN,NaN,NaN,2855/4958,2694/4221,898/1406,N/K,aaT/aaG,NaN,NaN,1,SNV,HGNC,HGNC:2343,YES,NM_201253.3,NaN,ENSP00000356370,P82279.210,A0A7D6VM04.8,UPI0000073345,P82279-1,NM_201253.3,deleterious(0),benign(0.204),Gene3D:2.10.25.10&AFDB-ENSP_mappings:AF-P82279...,NaN,NaN,NaN,-4.100,0,CRB1,23418,NM_201253,"[P82279-1, A0A7D6VM04]",/ibex/scratch/projects/c2102/databases/alphafo...,898,Asn,/ibex/scratch/projects/c2102/databases/dssp/al...,"[914, 917, 921, 922, 923, 924, 925]",False,78.600006,T,91,0.097497,0.658705,5.965850e-01,-0.087486,-6.183120e-02,-0.394596,-0.590773,0.029495,-0.287550,0.260123,0.0,0.0,0.0,-0.038429,-0.131667,1.325550e-02,0.0,0.00000,0.0,0.0,0.000000,0.0,Pathogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,197429467,1963628,G,A,NaN,NaN,NaN,2017299,Retinitis_pigmentosa_12|Leber_congenital_amaur...,NaN,"MONDO:MONDO:0010818,MedGen:C1838647,OMIM:60010...",NaN,NC_000001.11:g.197429467G>A,"criteria_provided,_single_submitter",Likely_pathogenic,NaN,NaN,single_nucleotide_variant,SO:0001483,NaN,NaN,"SO:0001583|missense_variant,SO:0001619|non-cod...",1,NaN,A,missense_variant,MODERATE,CRB1,ENSG00000134376,Transcript,ENST00000367400,pr

In [17]:
concatenated_df[['REFAA', 'ALTAA']] = concatenated_df['Amino_acids'].str.split('/', expand=True)


In [18]:
## replace concatenated_df.secondary_structure == 'nan' with 'L'
concatenated_df.secondary_structure = concatenated_df.secondary_structure.replace(np.nan, 'L')
concatenated_df.value_counts('secondary_structure')

secondary_structure
H    15891
L    14955
E     7454
T     4017
S     3299
G     1357
B      371
I      330
Name: count, dtype: int64

# **3. Normalize accesibility**

In [19]:
# Dictionary mapping amino acids to their respective x values
x_values = {
    'Ala': 129.0,
    'Arg': 274.0,
    'Asn': 195.0,
    'Asp': 193.0,
    'Cys': 167.0,
    'Glu': 223.0,
    'Gln': 225.0,
    'Gly': 104.0,
    'His': 224.0,
    'Ile': 197.0,
    'Leu': 201.0,
    'Lys': 236.0,
    'Met': 224.0,
    'Phe': 240.0,
    'Pro': 159.0,
    'Ser': 155.0,
    'Thr': 172.0,
    'Trp': 285.0,
    'Tyr': 263.0,
    'Val': 174.0
}

# Create the 'ACCESIBILITY_NORMALIZED' column
concatenated_df['ACCESIBILITY_NORMALIZED'] = concatenated_df.apply(lambda row: row['accessibility'] / x_values[row['Residue']], axis=1)


# **4. Save**

In [20]:
concatenated_df.to_csv("../results/1_merged.csv", index=False)
concatenated_df.to_pickle("../results/1_merged.pkl")